In [2]:
!pip install chromadb

In [3]:
import chromadb
from typing import Dict, List, Any, Optional
import logging

In [4]:
class ChromaDBManager:
    """
    Una clase de gestión robusta para operaciones CRUD de ChromaDB con manejo de errores mejorado,
    validación y soporte para operaciones masivas.
    """

    def __init__(self, path: str = "./data", collection_name: str = "Documents"):
        """
        Inicializa el gestor de ChromaDB con almacenamiento persistente.

        Args:
            path: Ruta para almacenar la base de datos.
            collection_name: Nombre de la colección a utilizar.
        """
        try:
            self.client = chromadb.PersistentClient(path=path)
            self.collection = self.client.get_or_create_collection(collection_name)
            self.logger = self._setup_logging()
        except Exception as e:
            raise RuntimeError(f"No se pudo inicializar ChromaDB: {str(e)}")

    def _setup_logging(self) -> logging.Logger:
        """Configura el registro de logs para el gestor de ChromaDB."""
        logger = logging.getLogger('ChromaDBManager')
        logger.setLevel(logging.INFO)
        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            logger.addHandler(handler)
        return logger

    def _validate_metadata(self, metadata: Dict[str, Any]) -> bool:
        """
        Valida la estructura de los metadatos.

        Args:
            metadata: Diccionario de metadatos a validar.

        Returns:
            bool: True si es válido, lanza ValueError si es inválido.
        """
        required_fields = {'categoria'}  # Agrega aquí los campos requeridos
        if not all(field in metadata for field in required_fields):
            raise ValueError(f"Los metadatos deben contener todos los campos requeridos: {required_fields}")
        return True

    def create_document(self, doc_id: str, text: str, metadata: Dict[str, Any]) -> bool:
        """
        Crea un nuevo documento con validación y manejo de errores.

        Args:
            doc_id: Identificador único para el documento.
            text: Contenido del documento.
            metadata: Metadatos asociados.

        Returns:
            bool: True si se crea exitosamente.
        """
        try:
            # Valida los metadatos
            self._validate_metadata(metadata)

            # Verifica si el documento ya existe
            existing = self.collection.get(ids=[doc_id])
            if existing["ids"]:
                raise ValueError(f"El documento con ID '{doc_id}' ya existe")

            self.collection.add(
                documents=[text],
                metadatas=[metadata],
                ids=[doc_id]
            )
            self.logger.info(f"Documento creado exitosamente: {doc_id}")
            return True

        except Exception as e:
            self.logger.error(f"Error al crear el documento: {str(e)}")
            raise

    def create_documents_bulk(self, documents: List[Dict[str, Any]]) -> bool:
        """
        Crea múltiples documentos en lote.

        Args:
            documents: Lista de diccionarios que contienen doc_id, texto y metadatos.

        Returns:
            bool: True si se crean exitosamente.
        """
        try:
            ids = []
            texts = []
            metadatas = []

            for doc in documents:
                self._validate_metadata(doc['metadata'])
                ids.append(doc['doc_id'])
                texts.append(doc['text'])
                metadatas.append(doc['metadata'])

            self.collection.add(
                documents=texts,
                metadatas=metadatas,
                ids=ids
            )
            self.logger.info(f"Creación masiva exitosa: {len(documents)} documentos")
            return True

        except Exception as e:
            self.logger.error(f"Error en la creación masiva: {str(e)}")
            raise

    def read_document(self, doc_id: str) -> Dict[str, Any]:
        """
        Lee un documento por su ID con manejo de errores mejorado.

        Args:
            doc_id: Identificador del documento.

        Returns:
            Dict con los datos y metadatos del documento.
        """
        try:
            result = self.collection.get(ids=[doc_id])
            if not result["documents"]:
                raise ValueError(f"No se encontró un documento con ID: {doc_id}")

            return {
                "document": result["documents"][0],
                "metadata": result["metadatas"][0]
            }

        except Exception as e:
            self.logger.error(f"Error al leer el documento: {str(e)}")
            raise

    def update_document(self, doc_id: str, new_text: Optional[str] = None,
                        new_metadata: Optional[Dict[str, Any]] = None) -> bool:
        """
        Actualiza un documento con soporte para actualizaciones parciales.

        Args:
            doc_id: Identificador del documento.
            new_text: Nuevo contenido del documento (opcional).
            new_metadata: Nuevos metadatos (opcional).

        Returns:
            bool: True si se actualiza exitosamente.
        """
        try:
            # Obtiene el documento existente
            existing = self.collection.get(ids=[doc_id])
            if not existing["documents"]:
                raise ValueError(f"No se encontró un documento con ID: {doc_id}")

            # Prepara los datos para actualizar
            text = new_text if new_text is not None else existing["documents"][0]
            metadata = new_metadata if new_metadata is not None else existing["metadatas"][0]

            # Valida los nuevos metadatos si se proporcionan
            if new_metadata is not None:
                self._validate_metadata(metadata)

            # Realiza la actualización
            self.collection.delete(ids=[doc_id])
            self.collection.add(
                documents=[text],
                metadatas=[metadata],
                ids=[doc_id]
            )
            self.logger.info(f"Documento actualizado exitosamente: {doc_id}")
            return True

        except Exception as e:
            self.logger.error(f"Error al actualizar el documento: {str(e)}")
            raise

    def delete_document(self, doc_id: str) -> bool:
        """
        Elimina un documento con verificación.

        Args:
            doc_id: Identificador del documento.

        Returns:
            bool: True si se elimina exitosamente.
        """
        try:
            # Verifica que el documento exista
            existing = self.collection.get(ids=[doc_id])
            if not existing["documents"]:
                raise ValueError(f"No se encontró un documento con ID: {doc_id}")

            self.collection.delete(ids=[doc_id])
            self.logger.info(f"Documento eliminado exitosamente: {doc_id}")
            return True

        except Exception as e:
            self.logger.error(f"Error al eliminar el documento: {str(e)}")
            raise

    def query_similar(self, text: str, top_k: int = 3,
                      filters: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]]:
        """
        Consulta documentos similares con manejo mejorado de resultados.

        Args:
            text: Texto de consulta.
            top_k: Número de resultados a devolver.
            filters: Filtros opcionales de metadatos.

        Returns:
            Lista de diccionarios con documentos similares y sus metadatos.
        """
        try:
            results = self.collection.query(
                query_texts=[text],
                n_results=top_k,
                where=filters
            )

            similar_docs = []
            if results["documents"] and results["metadatas"]:
                for docs, metas in zip(results["documents"], results["metadatas"]):
                    for doc, meta in zip(docs, metas):
                        similar_docs.append({
                            "text": doc,
                            "metadata": meta
                        })

            return similar_docs

        except Exception as e:
            self.logger.error(f"Error al consultar documentos similares: {str(e)}")
            raise

    def query_by_filter(self, filter_conditions: Dict[str, Any]) -> List[Dict[str, Any]]:
        """
        Consulta documentos por filtro con manejo mejorado de resultados.

        Args:
            filter_conditions: Condiciones de filtro de metadatos.

        Returns:
            Lista de diccionarios con documentos que coinciden.
        """
        try:
            results = self.collection.get(where=filter_conditions)

            filtered_docs = []
            for doc, meta, doc_id in zip(results["documents"], results["metadatas"], results["ids"]):
                filtered_docs.append({
                    "id": doc_id,
                    "text": doc,
                    "metadata": meta
                })

            return filtered_docs

        except Exception as e:
            self.logger.error(f"Error al consultar por filtro: {str(e)}")
            raise


In [6]:
# Ejemplo de uso
if __name__ == "__main__":
    # Inicializar el gestor
    db_manager = ChromaDBManager()

    try:
        # Crear un documento único
        db_manager.create_document(
            "doc1",
            "Este es un ejemplo de texto.",
            {"categoria": "ejemplo"}
        )

        # Crear documentos en lote
        bulk_docs = [
            {
                "doc_id": "doc2",
                "text": "ChromaDB es útil para manejar datos vectoriales.",
                "metadata": {"categoria": "tecnologia"}
            },
            {
                "doc_id": "doc3",
                "text": "Los modelos de lenguaje generan embeddings para el texto.",
                "metadata": {"categoria": "machine learning"}
            }
        ]
        db_manager.create_documents_bulk(bulk_docs)

        # Leer un documento
        doc = db_manager.read_document("doc1")
        print("Documento leído:", doc)

        # Actualizar un documento
        db_manager.update_document(
            "doc1",
            new_metadata={"categoria": "actualizado"}
        )

        # Consultar documentos similares
        similar = db_manager.query_similar("¿Cómo funciona ChromaDB?", top_k=2)
        print("Documentos similares:", similar)

        # Consultar documentos por filtro
        filtered = db_manager.query_by_filter({"categoria": "machine learning"})
        print("Documentos filtrados:", filtered)

        # Eliminar un documento
        db_manager.delete_document("doc1")

    except Exception as e:
        print(f"Error en el ejemplo de uso: {str(e)}")


2025-01-15 03:04:54,808 - ChromaDBManager - INFO - Documento creado exitosamente: doc1
INFO:ChromaDBManager:Documento creado exitosamente: doc1
2025-01-15 03:04:55,094 - ChromaDBManager - INFO - Creación masiva exitosa: 2 documentos
INFO:ChromaDBManager:Creación masiva exitosa: 2 documentos
2025-01-15 03:04:55,257 - ChromaDBManager - INFO - Documento actualizado exitosamente: doc1
INFO:ChromaDBManager:Documento actualizado exitosamente: doc1


Documento leído: {'document': 'Este es un ejemplo de texto.', 'metadata': {'categoria': 'ejemplo'}}


2025-01-15 03:04:55,429 - ChromaDBManager - INFO - Documento eliminado exitosamente: doc1
INFO:ChromaDBManager:Documento eliminado exitosamente: doc1


Documentos similares: [{'text': 'ChromaDB es útil para manejar datos vectoriales.', 'metadata': {'categoria': 'tecnologia'}}, {'text': 'Este es un ejemplo de texto.', 'metadata': {'categoria': 'actualizado'}}]
Documentos filtrados: [{'id': 'doc3', 'text': 'Los modelos de lenguaje generan embeddings para el texto.', 'metadata': {'categoria': 'machine learning'}}]
